# Demand Prediction
### Resources: 
- **Prohet**: 
    - https://machinelearningmastery.com/time-series-forecasting-with-prophet-in-python/
    - https://facebook.github.io/prophet/docs/seasonality,_holiday_effects,_and_regressors.html#additional-regressors
    - https://facebook.github.io/prophet/docs/quick_start.html
- **Recursive autoregressive forecast**:
    - https://www.cienciadedatos.net/documentos/py27-time-series-forecasting-python-scikitlearn.html
- **Neural Prophet**: 
    - https://www.kaggle.com/code/ohseokkim/predicting-future-by-lstm-prophet-neural-prophet
    - https://bobrupakroy.medium.com/neural-prophet-for-time-series-27bfeeaa5c7d

### Import libraries

In [3]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import pickle

# Prophet
from fbprophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly,plot_forecast_component, plot_yearly

# Neural Prophet
from neuralprophet import NeuralProphet

# Recursive autoregressive forecasting
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from sklearn.ensemble import RandomForestRegressor
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster

### Import data

In [4]:
itinerarios = pd.read_csv('../Data/Itinerarios/itinerarios_bases.csv',index_col=0, dtype={'zip_code': 'str'})

# No incluimos en el modelo de predicción los movimientos realizados por empleados de bicimad (user_type==3)
itinerarios1 = itinerarios[itinerarios["user_type"]!= 3]
# Solo incluimos en el modelo aquellos movimientos que duran más de 5 minutos (no se ha cogido una bici defectuosa) y menos de 1 dia (no se ha perdido)
itinerarios1 = itinerarios1[itinerarios1["status"] == "short_rental"]

# Contamos demanda por hora y día
itinerarios2 = itinerarios1[["unplug_hourTime"]]
itinerarios3 = itinerarios2.groupby("unplug_hourTime").size().reset_index(name='Count')
itinerarios3["unplug_hourTime"] = pd.to_datetime(itinerarios3["unplug_hourTime"])
itinerarios4 = itinerarios3[itinerarios3["unplug_hourTime"].dt.tz_localize(None) >= pd.to_datetime('20190801')]
itinerarios4.head()

,unplug_hourTime,Count
259,2019-08-01 00:00:00+00:00,136
260,2019-08-01 01:00:00+00:00,74
261,2019-08-01 02:00:00+00:00,66
262,2019-08-01 03:00:00+00:00,52
263,2019-08-01 04:00:00+00:00,100


En itinerarios5 se guarda el df de la demanda por día y hora, añadiendo las variables dayofweek, typeofday, hour

In [5]:
# Day of week Monday=0, Sunday=6.
itinerarios5 = itinerarios4.copy()
itinerarios5["dayofweek"]  = itinerarios5["unplug_hourTime"].dt.dayofweek
itinerarios5["hour"]  = itinerarios5["unplug_hourTime"].dt.hour
conditions = [
    (itinerarios5['dayofweek'] <= 3),
    (itinerarios5['dayofweek'] == 4),
    (itinerarios5['dayofweek'] == 5) | (itinerarios5['dayofweek'] == 6)
    ]
values = ['workingday', 'friday', 'weekend']

itinerarios5["typeofday"]  = np.select(conditions, values)
itinerarios5.head()

,unplug_hourTime,Count,dayofweek,hour,typeofday
259,2019-08-01 00:00:00+00:00,136,3,0,workingday
260,2019-08-01 01:00:00+00:00,74,3,1,workingday
261,2019-08-01 02:00:00+00:00,66,3,2,workingday
262,2019-08-01 03:00:00+00:00,52,3,3,workingday
263,2019-08-01 04:00:00+00:00,100,3,4,workingday


itinerarios6 es el df usado para predecir con prophet. Las variables del tiempo y la demanda se convierten en ds e y, repsecitvamente para que pueda ser procesado por el modelo

In [6]:
itinerarios6 = itinerarios4.copy()
itinerarios6.columns = ['ds', 'y']
#itinerarios6["ds"] = pd.to_datetime(itinerarios6["ds"])
itinerarios6["ds"] = itinerarios6["ds"].dt.tz_localize(None)
itinerarios6.head()

,ds,y
259,2019-08-01 00:00:00,136
260,2019-08-01 01:00:00,74
261,2019-08-01 02:00:00,66
262,2019-08-01 03:00:00,52
263,2019-08-01 04:00:00,100


### Prophet